## Cálculo da emissão e distância percorrida 

Nesse notebook é calculada a distância percorrida entre os pontos e a emissão de poluentes

In [1]:
import pandas as pd
import random
import numpy as np
import random
import geopandas as gpd
from utils.save_csv import save_csv
from utils.load_csv import load_csv
from datetime import datetime
from pyproj import Transformer

In [2]:
consumo_diesel = load_csv('consumo_diesel.csv')

df_posicoes = load_csv('df_posicoes.csv')

In [3]:
gdf = gpd.GeoDataFrame(df_posicoes, geometry=gpd.points_from_xy(df_posicoes.lon, df_posicoes.lat), crs="EPSG:4326")

gdf = gdf.to_crs("EPSG:31983")

gdf

,linha,id_onibus,timestamp,lat,lon,is_eletrico,modelo,geometry
0,5010-10,68853,2025-10-02 12:48:43-03:00,-23.649045,-46.640405,False,Articulado (18m),POINT (332687.686 7383667.185)
1,5010-10,68383,2025-10-02 12:48:52-03:00,-23.662680,-46.631144,False,Midiônibus,POINT (333649.734 7382168.184)
2,5010-10,68142,2025-10-02 12:49:09-03:00,-23.642878,-46.656716,False,Padron,POINT (331015.815 7384330.966)
3,5010-10,68338,2025-10-02 12:49:12-03:00,-23.662680,-46.631144,False,Articulado (23m),POINT (333649.734 7382168.184)
4,5010-10,68362,2025-10-02 12:49:06-03:00,-23.644669,-46.673169,False,Básico,POINT (329339.667 7384113.118)
...,...,...,...,...,...,...,...,...
149815,4092-10,47857,2025-10-02 12:56:07-03:00,-23.580189,-46.419959,False,Padron,POINT (355099.596 7391532.745)
149816,7062-10,78133,2025-10-02 12:56:21-03:00,-23.641820,-46.766292,False,Midiônibus,POINT (319835.301 7384314.174)
149817,7053-22,78167,2025-10-02 12:55:46-03:00,-23.637100,-46.779115,False,Biarticulado,POINT (318520.615 7384820.6)
149818,4008-21,48384,2025-10-02 12:56:26-03:00,-23.578815,-46.432018,False,Padron,POINT (353867.385 7391672.635)


In [4]:
gdf["x"] = gdf.geometry.x
gdf["y"] = gdf.geometry.y

gdf.head()

,linha,id_onibus,timestamp,lat,lon,is_eletrico,modelo,geometry,x,y
0,5010-10,68853,2025-10-02 12:48:43-03:00,-23.649045,-46.640405,False,Articulado (18m),POINT (332687.686 7383667.185),332687.685799,7.383667e+06
1,5010-10,68383,2025-10-02 12:48:52-03:00,-23.662680,-46.631144,False,Midiônibus,POINT (333649.734 7382168.184),333649.734082,7.382168e+06
2,5010-10,68142,2025-10-02 12:49:09-03:00,-23.642878,-46.656716,False,Padron,POINT (331015.815 7384330.966),331015.814984,7.384331e+06
3,5010-10,68338,2025-10-02 12:49:12-03:00,-23.662680,-46.631144,False,Articulado (23m),POINT (333649.734 7382168.184),333649.734082,7.382168e+06
4,5010-10,68362,2025-10-02 12:49:06-03:00,-23.644669,-46.673169,False,Básico,POINT (329339.667 7384113.118),329339.666989,7.384113e+06


In [5]:
consumo_diesel['emissa_no2_l_diesel'] = [random.uniform(0.02, 0.05) for _ in range(len(consumo_diesel))]

consumo_diesel['emissa_co2_l_diesel'] = [random.uniform(0.05, 0.1) for _ in range(len(consumo_diesel))]

In [6]:
def get_tipo_onibus(id_onibus):

    onibus = df_posicoes[df_posicoes['id_onibus']==id_onibus]

    return onibus['modelo'].unique()[0]

In [7]:
def get_fator_consumo(tipo_veiculo):

    cosumo_modelo = consumo_diesel[consumo_diesel['tecnologia']==tipo_veiculo]

    return cosumo_modelo['com_ar_l_km'].unique()[0]

def get_consumo_co2(tipo_veiculo, km_rodados):

    consumo_l = get_fator_consumo(tipo_veiculo)
    consumo_fato = consumo_l * km_rodados

    fator_emissao_modelo = consumo_diesel[consumo_diesel['tecnologia']==tipo_veiculo]
    fator_emissao_co2_modelo = fator_emissao_modelo['emissa_co2_l_diesel'].unique()[0]
    emissao_co2 = fator_emissao_co2_modelo * consumo_fato

    return emissao_co2

def get_consumo_no2(tipo_veiculo, km_rodados):

    consumo_l = get_fator_consumo(tipo_veiculo)
    consumo_fato = consumo_l * km_rodados

    fator_emissao_modelo = consumo_diesel[consumo_diesel['tecnologia']==tipo_veiculo]
    fator_emissao_no2_modelo = fator_emissao_modelo['emissa_no2_l_diesel'].unique()[0]
    emissao_no2 = fator_emissao_no2_modelo * consumo_fato

    return emissao_no2

In [8]:
data_frame = []

#for id_onibus in gdf['id_onibus'].unique():
    #df_onibus = gdf[gdf['id_onibus'] == id_onibus].reset_index(drop=True)
    #df_onibus.sort_values(by='timestamp', inplace=True)

for id_onibus in gdf['id_onibus'].unique():
    df_onibus = (gdf[gdf['id_onibus'] == id_onibus].sort_values(by='timestamp').reset_index(drop=True))

    for i, row in df_onibus.iterrows():
 
        if i < len(df_onibus)-1:
            x_atual = row['x']
            x_depois = df_onibus.loc[i+1, 'x']
            y_atual = row['y']
            y_depois = df_onibus.loc[i+1, 'y']
            momento_inicial = row['timestamp']
            momento_final = df_onibus.loc[i+1, 'timestamp']
            tipo_veiculo = get_tipo_onibus(row['id_onibus'])
            litro_diesel = get_fator_consumo(tipo_veiculo)
            
            dist = np.sqrt((x_depois - x_atual)**2 + (y_depois - y_atual)**2)
            
            dados = {
                'id_onibus' : row['id_onibus'],
                'linha' : row['linha'],
                'is_eletrico' : row['is_eletrico'],
                'distancia_percorrida' : dist,
                'ponto_inicial' : [x_atual, y_atual],
                'ponto_final' : [x_depois, y_depois],
                'momento_inicial' : momento_inicial,
                'momento_final' : momento_final,
                'modelo' : tipo_veiculo,
                'litro_diesel_km' : litro_diesel,
                'emissao_no2' : get_consumo_no2(tipo_veiculo, dist),
                'emissao_co2' : get_consumo_co2(tipo_veiculo, dist)
            }

            data_frame.append(dados)

gdf_final = pd.DataFrame(data_frame)
gdf_final.head()

,id_onibus,linha,is_eletrico,distancia_percorrida,ponto_inicial,ponto_final,momento_inicial,momento_final,modelo,litro_diesel_km,emissao_no2,emissao_co2
0,68853,5010-10,False,148.349719,"[332687.68579879985, 7383667.184539581]","[332792.2284083655, 7383772.438907389]",2025-10-02 12:48:43-03:00,2025-10-02 12:49:30-03:00,Articulado (18m),0.8,3.281178,9.989606
1,68853,5010-10,False,32.385046,"[332792.2284083655, 7383772.438907389]","[332804.84353484295, 7383802.265905192]",2025-10-02 12:49:30-03:00,2025-10-02 12:50:12-03:00,Articulado (18m),0.8,0.716288,2.180751
2,68853,5010-10,False,0.000000,"[332804.84353484295, 7383802.265905192]","[332804.84353484295, 7383802.265905192]",2025-10-02 12:50:12-03:00,2025-10-02 12:50:12-03:00,Articulado (18m),0.8,0.000000,0.000000
3,68853,5010-10,False,87.364702,"[332804.84353484295, 7383802.265905192]","[332744.1297509116, 7383865.086503765]",2025-10-02 12:50:12-03:00,2025-10-02 12:51:22-03:00,Articulado (18m),0.8,1.932320,5.882984
4,68853,5010-10,False,51.326566,"[332744.1297509116, 7383865.086503765]","[332692.8276940262, 7383863.500523301]",2025-10-02 12:51:22-03:00,2025-10-02 12:51:41-03:00,Articulado (18m),0.8,1.135234,3.456240


In [9]:
gdf_final['momento_inicial'] = pd.to_datetime(gdf_final['momento_inicial'])

In [10]:
transformer = Transformer.from_crs("EPSG:31983", "EPSG:4326", always_xy=True)

dados_trips_layer = []

for onibus in gdf_final['id_onibus'].unique():
    df_onibus = gdf_final[gdf_final['id_onibus'] == onibus].reset_index(drop=True)
    coordinates = []
    timestamps = []
    emissao_total = 0
    df_onibus.sort_values(by='momento_inicial', inplace=True)
    
    for i, row in df_onibus.iterrows():
        x, y = row['ponto_inicial']  
        lon, lat = transformer.transform(x, y) 
        
        coordinates.append([lon, lat])
        stamp = datetime.fromisoformat(row['momento_inicial'].isoformat())
        timestamps.append(stamp)
        emissao_total += row['emissao_co2']
    
    dados_trips_layer.append({
        'id_onibus': onibus,
        'linha': row['linha'],
        'emissao_co2': emissao_total,
        'is_eletrico': row['is_eletrico'],
        'coordinates' : coordinates, 
        'timestamps' : timestamps
    })

df_trips = pd.DataFrame(dados_trips_layer)

In [11]:
def init_on_zero(timestamp_list)->list[float]:
    
    inicial = timestamp_list[0]

    new_timestamps = [ts - inicial for ts in timestamp_list]

    return [int(ts.total_seconds()) for ts in new_timestamps]

df_trips['timestamps'] = df_trips['timestamps'].apply(init_on_zero)

In [12]:
coord_t = str(df_trips.loc[0, 'coordinates'])

In [13]:
eval(coord_t)

[[-46.640405, -23.649045499999996],
 [-46.6393685, -23.648105999999995],
 [-46.6392415, -23.647838],
 [-46.6392415, -23.647838],
 [-46.6398295, -23.6472645],
 [-46.6403325, -23.647273499999997],
 [-46.6403325, -23.647273499999997],
 [-46.641467000000006, -23.646593999999993],
 [-46.642595, -23.646401999999995],
 [-46.644012, -23.646169750000002],
 [-46.644012, -23.646169750000002],
 [-46.644012, -23.646169750000002],
 [-46.6444255, -23.646103499999995],
 [-46.6444255, -23.646103499999995]]

In [14]:
df_trips.loc[0, 'timestamps']

[0, 47, 89, 89, 159, 178, 178, 225, 266, 311, 311, 355, 399, 412]

In [15]:
df_trips = df_trips[['coordinates', 'timestamps']]

In [16]:
save_csv(gdf_final, 'gdf_final.csv')

save_csv(df_trips, 'df_trips.csv')

Base salva em data\gdf_final.csv
Base salva em data\df_trips.csv
